<a href="https://colab.research.google.com/github/Bhavani-2005/DL-Project/blob/main/Image_caption_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
import matplotlib.pyplot as plt
import pickle
import json
from PIL import Image
import pandas as pd
import os
import numpy as np
import cv2



DataSet

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
!ls "/content/drive/MyDrive"


'Colab Notebooks'  'Image Caption generation'


In [22]:
import os

dataset_path = '/content/drive/MyDrive/Image Caption generation/DataSet'

# List the contents of dataset folder
print("Dataset folder contents:", os.listdir(dataset_path))

# Set correct paths
image_folder = os.path.join(dataset_path, 'Images')
caption_file = os.path.join(dataset_path, 'captions.txt')

# Verify the files exist
print("Images folder contains:", len(os.listdir(image_folder)), "files")
print("First 5 images:", os.listdir(image_folder)[:5])
print("Captions file exists:", os.path.isfile(caption_file))


Dataset folder contents: ['Images', 'captions.txt']
Images folder contains: 2372 files
First 5 images: ['1457762320_7fe121b285.jpg', '1466479163_439db855af.jpg', '146098876_0d99d7fb98.jpg', '1468429623_f001988691.jpg', '1448511770_1a4a9c453b.jpg']
Captions file exists: True


Preprocessing Captions

In [23]:
import string

# Load captions file
with open(caption_file, 'r') as f:
    captions_doc = f.read()

# Create dictionary mapping image names to their captions
captions_dict = {}

for line in captions_doc.strip().split('\n'):
    if len(line) < 1:
        continue
    img_cap_pair = line.split('\t')
    if len(img_cap_pair) == 1:
        img_cap_pair = line.split(' ')
        img_name = img_cap_pair[0].split('#')[0]
        caption = ' '.join(img_cap_pair[1:])
    else:
        img_name, caption = img_cap_pair
    img_name = img_name.split('#')[0]
    caption = caption.lower()
    caption = caption.translate(str.maketrans('', '', string.punctuation))
    caption = '<start> ' + caption + ' <end>'
    if img_name not in captions_dict:
        captions_dict[img_name] = []
    captions_dict[img_name].append(caption)

# Show example output
for img, caps in list(captions_dict.items())[:1]:
    print("Image:", img)
    print("Captions:")
    for c in caps:
        print("-", c)

Image: image,caption
Captions:
- <start>  <end>


Tokenizing Captions

In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle

# Gather all captions into a list
all_captions = []
for key in captions_dict:
    all_captions.extend(captions_dict[key])

# Initialize the tokenizer and fit on captions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary size:", vocab_size)

# Save the tokenizer for later use
tokenizer_path = '/content/drive/MyDrive/Image Caption generation/DataSet/tokenizer.pkl'
with open(tokenizer_path, 'wb') as f:
    pickle.dump(tokenizer, f)

print(f"Tokenizer saved at: {tokenizer_path}")


Vocabulary size: 8746
Tokenizer saved at: /content/drive/MyDrive/Image Caption generation/DataSet/tokenizer.pkl


Find Maximum Caption Length

In [25]:
# Find the maximum length among all captions
max_length = max(len(c.split()) for c in all_captions)

print("Maximum caption length:", max_length)


Maximum caption length: 37


Extracting Image Features

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
import numpy as np
import pickle
import os

# Load the InceptionV3 model without the final classification layer
base_model = InceptionV3(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)

# Path to images folder
image_folder = '/content/drive/MyDrive/Image Caption generation/DataSet/Images'

# Dictionary to hold image features
features = {}

# Process and extract features for each image
for img_name in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_name)
    # Load the image with target size expected by InceptionV3
    image = load_img(img_path, target_size=(299, 299))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    # Extract features
    feature = model.predict(image, verbose=0)
    features[img_name] = feature

# Save the features to a file
features_path = '/content/drive/MyDrive/Image Caption generation/DataSet/features.pkl'
with open(features_path, 'wb') as f:
    pickle.dump(features, f)

print(f"Extracted features for {len(features)} images and saved to {features_path}")


96112376/96112376 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
